# Speech command recognition - Day 3 Part 2
## Model improvement
******
Author: Duowei Tang \
Reference: This exercise is adopted from a Pytorch tutorial: https://pytorch.org/tutorials/intermediate/speech_command_classification_with_torchaudio_tutorial.html \
Pytorch ignite: https://pytorch-ignite.ai/ \
The Aachen RIR dataset: https://www.iks.rwth-aachen.de/en/research/tools-downloads/databases/aachen-impulse-response-database/

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
from scipy.io import wavfile

import matplotlib.pyplot as plt
import IPython.display as ipd

from torchaudio.datasets import SPEECHCOMMANDS
import glob
import random
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np

from ignite.engine import Engine, Events
from ignite.metrics import Accuracy, Loss, Precision, Recall, confusion_matrix
from ignite.handlers import ModelCheckpoint, LRScheduler
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine, ProgressBar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

### Class lables
*******
In the code block below, it listed the selected command classes and all class labels that are commented out are considered as "unknown".

In [ ]:
labels = ['forward', 'backward', 'up', 'down',
          'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero',
          'left', 'right', 'go', 'stop', 'yes', 'no', 'on', 'off', 'unknown']
# The following dataset labels are considered unkonwn
# unknown = ['bed', 'bird', 'cat', 'dog', 'follow', 'happy', 'house', 'learn', 'marvin',
#            'sheila', 'visual', 'wow', 'tree']

### Prepare the datasets
*******
Unlike the previous day where we first store the extracted MFCC features to the disk and load the pre-computed features during training. In this exercise, we will extract the features on-the-fly during training.

In [ ]:
SPEECH_DATA_ROOT = ******The root address where to store the dataset*******
class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__(os.path.dirname(SPEECH_DATA_ROOT), download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:
                return [os.path.normpath(os.path.join(self._path, line.strip())) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]

# Create the paritions and features will be extracted during training
train_set = SubsetSC("training")
valid_set = SubsetSC("validation")
test_set = SubsetSC("testing")

In [ ]:
def label_to_index(word):
    if word in labels:
        return torch.tensor(labels.index(word))
    else:
        return torch.tensor(labels.index("unknown"))

def index_to_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return labels[index]

### Create a data sampler
******

In [ ]:
# The training data per class count is as follows, the code to calculate this is in the appendix
train_count = torch.tensor([ 1251.,  1342.,  2935.,  3121.,  3127.,  3093.,  2956.,  2942.,  3221.,
         3074.,  3190.,  3019.,  3158.,  3237.,  3022.,  3006.,  3091.,  3099.,
         3221.,  3121.,  3076.,  2951., 20227.])
# Customize a weighted random sampler for the training set
class WeightedRandomSampler(torch.utils.data.Sampler):
    """Samples elements from [0,..,len(weights)-1] with given probabilities (weights).

    Args:
        weights (list) : a list of weights, not necessary summing up to one
        num_samples (int) : number of samples to draw
        replacement (bool): if True, samples are drawn with replacement.

    """

    def __init__(self, train_count, num_samples, replacement=True):
        actual_dist = train_count / torch.sum(train_count)

        desired_dist = torch.tensor([1/len(labels)] * len(labels))
        self.per_class_weights = desired_dist / actual_dist
        self.num_samples = num_samples
        self.replacement = replacement

        # Assign a weight to each example
        # Check if the weights is saved in the disk to save time
        if os.path.exists("weights_per_sample.pt"):
            self.weights = torch.load("weights_per_sample.pt")
        else:
            self.weights = torch.tensor([self.per_class_weights[label_to_index(i)] for _, _, i, *_ in train_set])
            torch.save(self.weights, "weights_per_sample.pt")

    def __iter__(self):
        return iter(torch.multinomial(self.weights, self.num_samples, self.replacement))

    def __len__(self):
        return self.num_samples

In [ ]:
sample_rate = 16000

class MFCC_with_freq_masking(torchaudio.transforms.MFCC):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.freq_masking = torchaudio.transforms.FrequencyMasking(freq_mask_param=5)

    def forward(self, waveform):
        mel_specgram = self.MelSpectrogram(waveform)
        mel_specgram = self.freq_masking(mel_specgram)
        if self.log_mels:
            log_offset = 1e-6
            mel_specgram = torch.log(mel_specgram + log_offset)
        else:
            mel_specgram = self.amplitude_to_DB(mel_specgram)

        # (..., time, n_mels) dot (n_mels, n_mfcc) -> (..., n_nfcc, time)
        mfcc = torch.matmul(mel_specgram.transpose(-1, -2), self.dct_mat).transpose(-1, -2)
        return mfcc
    
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)

def extract_mfcc_train(waveform):
    mfcc = MFCC_with_freq_masking(
        sample_rate=sample_rate,
        n_mfcc=16,
        melkwargs={"n_fft": int(0.03*sample_rate), "hop_length": int(0.03*0.5*sample_rate), "n_mels": 64,
                   "window_fn": torch.hamming_window, "center": False, "pad_mode": "reflect"},
    )
    return mfcc(waveform)

def extract_mfcc(waveform):
    mfcc = torchaudio.transforms.MFCC(
        sample_rate=sample_rate,
        n_mfcc=16,
        melkwargs={"n_fft": int(0.03*sample_rate), "hop_length": int(0.03*0.5*sample_rate), "n_mels": 64,
                   "window_fn": torch.hamming_window, "center": False, "pad_mode": "reflect"},
    )
    return mfcc(waveform)

# Preload the noises to save time
noise_files = glob.glob(os.path.join(SPEECH_DATA_ROOT, "speech_commands_v0.02", "_background_noise_", "*.wav"))
noise_waveforms = [torchaudio.load(noise_file)[0] for noise_file in noise_files]
def add_exist_noise(waveform):
    # Apply noise
    noise_waveform = noise_waveforms[torch.randint(0, len(noise_waveforms), (1,)).item()]
    # Make waveform and noise the same length by padding with zeros
    if noise_waveform.size(-1) < waveform.size(-1):
        noise_waveform = F.pad(noise_waveform, (0, waveform.size(-1) - noise_waveform.size(-1)))
    elif noise_waveform.size(-1) >= waveform.size(-1):
        # randomly crop noise
        max_offset = noise_waveform.size(-1) - waveform.size(-1)
        offset = torch.randint(0, max_offset, (1,))
        noise_waveform = noise_waveform[..., offset:offset+waveform.size(-1)]
    waveform = torchaudio.transforms.AddNoise()(waveform, noise_waveform, snr=torch.randint(-5, 10, (1,)))
    return waveform

def perturbate_speed(waveform):
    waveform_aug = torchaudio.transforms.SpeedPerturbation(orig_freq=sample_rate, factors=[0.9, 1.1, 1.0, 1.0])(waveform)[0]
    if waveform_aug.size(-1) < waveform.size(-1):
        waveform_aug = F.pad(waveform_aug, (0, waveform.size(-1) - waveform.size(-1)))
    else:
        length_diff = waveform_aug.size(-1) - waveform.size(-1)
        waveform_aug = waveform_aug[..., length_diff//2:length_diff//2+waveform.size(-1)]
    return waveform_aug

# Data Augmentation
def data_augment(waveform):
    augmentations = [
        torchaudio.transforms.Vol(gain=torch.randint(low=-10, high=10, size=(1,)), gain_type='db'),
        torchaudio.transforms.TimeMasking(time_mask_param=int(0.05*sample_rate), p=1.0),
        add_exist_noise,
        perturbate_speed,
    ]
    selected_augmentations = random.sample(augmentations, 1)
    for augmentation in selected_augmentations:
        waveform = augmentation(waveform)
    return waveform

def collate_fn_extract_feature_train(batch):
    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number
    tensors, targets = [], []
    # Gather in lists, and encode labels as indices
    for waveform, sample_rate, label, speaker_id, utterance_number in batch:
        # Apply data augmentation
        waveform = data_augment(waveform)
        tensors += [waveform]
        targets += [label_to_index(label)]
    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    # Extract MFCC features (with frequency masking included)
    tensors = extract_mfcc_train(tensors)
    # Squeeze and permute 
    tensors = tensors.squeeze(1).permute(0, 2, 1)
    targets = torch.stack(targets)

    return tensors, targets

def collate_fn_extract_feature_eval(batch):
    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number
    tensors, targets = [], []
    # Gather in lists, and encode labels as indices
    for waveform, sample_rate, label, speaker_id, utterance_number in batch:
        tensors += [waveform]
        targets += [label_to_index(label)]
    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    # Extract MFCC features
    tensors = extract_mfcc(tensors)
    # Squeeze and permute 
    tensors = tensors.squeeze(1).permute(0, 2, 1)
    targets = torch.stack(targets)

    return tensors, targets

batch_size = 512
num_workers = 8

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    # shuffle=True,
    drop_last=True,
    sampler=WeightedRandomSampler(train_count, num_samples=len(train_set), replacement=True),
    collate_fn=collate_fn_extract_feature_train,
    num_workers=num_workers,
    pin_memory=False,
    multiprocessing_context='fork',
    persistent_workers=True,
)
# A separate loader for train evaluation without data augmentation
train_eval_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn_extract_feature_eval,
    num_workers=num_workers,
    pin_memory=False,
    multiprocessing_context='fork',
    persistent_workers=True,
)
valid_loader = torch.utils.data.DataLoader(
    valid_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn_extract_feature_eval,
    num_workers=num_workers,
    pin_memory=False,
    multiprocessing_context='fork',
    persistent_workers=True,
)

### Model design
******

In [ ]:
# A bigger model with LSTM
class CNN_LSTM(nn.Module):
    def __init__(self, n_input=16, n_output=23, n_channel=32):
        super().__init__()
        self.cnn1 = nn.Conv1d(n_input, n_channel, kernel_size=16)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(2)
        self.cnn2 = nn.Conv1d(n_channel, 2*n_channel, kernel_size=8)
        self.bn2 = nn.BatchNorm1d(2*n_channel)
        self.pool2 = nn.MaxPool1d(2)
        # self.conv3 = nn.Conv1d(2*n_channel, 4 * n_channel, kernel_size=3)
        # self.bn3 = nn.BatchNorm1d(4 * n_channel)
        # self.pool3 = nn.MaxPool1d(3)
        self.lstm = nn.LSTM(2*n_channel, 2*n_channel, num_layers=1, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(4 * n_channel, n_output)

    def forward(self, x):
        x = torch.permute(x, [0, 2, 1])
        x = F.relu(self.bn1(self.cnn1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.cnn2(x)))
        x = self.pool2(x)
        # x = F.relu(self.bn3(self.conv3(x)))
        # x = self.pool3(x)
        x = self.dropout(x)
        x = torch.permute(x, [0, 2, 1])
        x, (h_n, c_n) = self.lstm(x)
        x = x[:, -1, :]
        x = x.squeeze(1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)

model = CNN_LSTM(n_input=16, n_output=len(labels))
model.to(device)
print(model)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

n = count_parameters(model)
print("Number of parameters: %s" % n)

### Set up the optimizer and learning rate scheduler
******

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
torch_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)  # reduce the learning after 20 epochs by a factor of 10
scheduler = LRScheduler(torch_lr_scheduler)

### Training and validation using Ignite
******

In [ ]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()

def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)

In [ ]:
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch
    x, y = x.to(device), y.to(device)
    y_pred = model(x)
    loss = F.nll_loss(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)
ProgressBar().attach(trainer)

In [ ]:
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        return y_pred, y
    
train_evaluator = Engine(validation_step)
valid_evaluator = Engine(validation_step)

### Attach evaluation metrics
******
We will log the loss (i.e. NLL), accuracy, precision, recall confusion matrix metrics to the training and evaluation evaluators. Then we will visualize those quantities in Tensorboard.

In [ ]:
# Attach all the evaluation metrics to the evaluators
def attach_metrics(evaluator):
      Loss(F.nll_loss).attach(evaluator, "nll")
      Accuracy().attach(evaluator, "accuracy")
      my_recall = Recall(average=True)
      my_recall.attach(evaluator, "recall")
      my_precision = Precision(average=True)
      my_precision.attach(evaluator, "precision")
      
      my_recall = Recall(average=False)
      my_precision = Precision(average=False)
      f1 = (my_precision * my_recall * 2 / (my_precision + my_recall)).mean()
      f1.attach(evaluator, "f1")

      confusion = confusion_matrix.ConfusionMatrix(num_classes=len(labels))
      confusion.attach(evaluator, "cm")      

attach_metrics(train_evaluator)
attach_metrics(valid_evaluator)

In [ ]:
def generate_CM_png(cm):
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    # Calculate accuracy and put in the title
    accuracy = np.trace(cm.to_numpy()) / np.sum(cm.to_numpy())
    fig = plt.figure(figsize=(25, 10))
    fig.tight_layout()
    # Normalize the confusion matrix
    cm = cm / (np.sum(cm.to_numpy(), axis=1) + 1e-10)
    sn.heatmap(cm, annot=True, cmap='Blues')
    # Put x-axis label and y-axis label
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("Accuracy: {:.1f}%".format(accuracy*100))
    return fig

In [ ]:
def tb_log(model, trainer, evaluator, tag, root_dir="./day3_tb_logs_part2_LSTM"):
    # Define a Tensorboard logger
    tb_logger = TensorboardLogger(log_dir=os.path.join(root_dir, tag))

    # Attach the logger to log learning rate
    tb_logger.attach_opt_params_handler(
        trainer,
        event_name=Events.ITERATION_STARTED,
        optimizer=optimizer
    )
    # Attach handler for plotting both evaluators' metrics after every epoch completes
    tb_logger.attach_output_handler(
        evaluator,
        event_name=Events.EPOCH_COMPLETED,
        tag="",
        metric_names=["nll", "accuracy", "recall", "precision", "f1"],
        global_step_transform=global_step_from_engine(trainer),
    )
    # Attach handler to plot the confusion matrix after every epoch completes
    @evaluator.on(Events.EPOCH_COMPLETED)
    def image_logger():
        metrics = evaluator.state.metrics
        cm = metrics["cm"]
        res = generate_CM_png(cm)
        global_step = global_step_from_engine(trainer)(evaluator, Events.EPOCH_COMPLETED)
        tb_logger.writer.add_figure(tag=tag, figure=res, global_step=global_step)

# Define the training actions
validate_every = 1
# It may need more epochs because of the data augmentation
max_epochs = 80
# Evaluate on the training set every validate_every epochs
@trainer.on(Events.EPOCH_COMPLETED(every=validate_every))
def run_train_eval():
    train_evaluator.run(train_eval_loader)

# Evaluate on the validation set every validate_every epochs
@trainer.on(Events.EPOCH_COMPLETED(every=validate_every))
def run_valid_eval():
    valid_evaluator.run(valid_loader)

# Save the model after every epoch
checkpointer = ModelCheckpoint(
    "./day3_tb_logs_part2_LSTM/day3_models", "", n_saved=max_epochs, create_dir=True, save_as_state_dict=True, require_empty=False
)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {"model": model})

# Attach the tensorboard logger
tb_log(model, trainer, train_evaluator, tag="training")
tb_log(model, trainer, valid_evaluator, tag="validation")
    
# Attach the learning rate scheduler to the trainer to adjust the learning rate
trainer.add_event_handler(Events.EPOCH_STARTED, scheduler)

# Run the trainer
trainer.run(train_loader, max_epochs)

### Test model on the testing set
******

In [ ]:
# Please select the model based on the validation metrics and fill in the model path here
test_epoch=70
model = CNN_LSTM(n_input=16, n_output=len(labels))
model.load_state_dict(torch.load(f"day3_tb_logs_part2_LSTM/day3_models/model_{test_epoch*165}.pt"))

In [ ]:
def predict(tensor):
    model.eval()
    # Use the model to predict the label of the waveform
    tensor = tensor.to(device)
    # tensor = transform(tensor)
    # if tensor shape is not 16000, then the waveform is too short and we need to pad it
    if tensor.numel() != 16000:
        tensor = F.pad(tensor, (0, 16000 - tensor.numel()), "constant", 0.0)

    # # Calculate MFCC
    tensor = extract_mfcc(tensor)
    tensor = torch.squeeze(tensor, (0, 1)).t()

    tensor = model(tensor.unsqueeze(0))
    tensor = get_likely_index(tensor)
    tensor = index_to_label(tensor.squeeze())
    return tensor

correct = 0
test_pred = []
test_target = []
for i, (waveform, sample_rate, label, speaker_id, utterance_number) in enumerate(test_set):
    output = predict(waveform)
    test_pred.append(output)
    test_target.append(label)
    if label in labels:
        if output == label:
            correct += 1
        else:
            print(f"Data point spk {speaker_id} utt {utterance_number}. Expected: {label}. Predicted: {output}.")
    else:
        if output == "unknown":
            correct += 1
        else:
            print(f"Data point spk {speaker_id} utt {utterance_number}. Expected: unknown. Predicted: {output}.")
        
print(f"Accuracy: {correct}/{len(test_set)} ({100. * correct / len(test_set):.0f}%)")

### Test with your own voice
******

In [ ]:
import sounddevice as sd
print(sd.query_devices())

In [ ]:
# Please chose the device id from the list of devices above, and fill in the device id here
# E.g. sd.default.device = "MacBook Pro Microphone"
sd.default.device = ***CHOSE THE DEVICE ID HERE***
def record(seconds=5, sample_rate=16000):
    # Make a 1s recording
    print("Start recording.")
    recording = sd.rec(int(seconds * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()
    
    # Define the file format
    fileformat = "wav"
    filename = f"_audio.{fileformat}"
    # Write the recording to a file using scipy wavfile
    wavfile.write(filename, sample_rate, recording)
    return torchaudio.load(filename)

# Detect whether notebook runs in google colab
record_wav, sample_rate = record()
# sample_rate, record_wav = wavfile.read("_audio.wav")
# Check if record_wav is a torch tensor
if not isinstance(record_wav, torch.Tensor):
    record_wav = torch.tensor(record_wav, dtype=torch.float32)
record_wav = torch.reshape(record_wav, (1, -1))
print(f"Predicted: {predict(record_wav)}.")
ipd.Audio(record_wav, rate=sample_rate)

### Code Appendix
*******

In [ ]:
# # Count the number of training examples per label
# labels_train = []
# for _, label in train_loader:
#     labels_train.append(label)

# labels_train = torch.stack(labels_train)
# labels_train = labels_train.view(-1)

# print("Shape of labels_train:", labels_train.size())

# # Count the number of training examples per label
# train_count = torch.bincount(labels_train).float()
# print("Count of labels:", train_count)

# # Plot the count of train examples per label
# plt.figure(figsize=(10, 5))
# plt.bar(torch.arange(len(train_count)), train_count.numpy())
# plt.xticks(torch.arange(len(train_count)), labels, rotation=45)
# plt.ylabel("Count")
# plt.xlabel("Label")
# plt.title("Number of training examples per label")
# plt.show()

In [ ]:
# A bigger model with LSTM
# class CNN(nn.Module):
#     def __init__(self, n_input=16, n_output=23, n_channel=32):
#         super().__init__()
#         self.cnn1 = nn.Conv1d(n_input, n_channel, kernel_size=16)
#         self.bn1 = nn.BatchNorm1d(n_channel)
#         self.pool1 = nn.MaxPool1d(2)
#         self.cnn2 = nn.Conv1d(n_channel, 2*n_channel, kernel_size=8)
#         self.bn2 = nn.BatchNorm1d(2*n_channel)
#         self.pool2 = nn.MaxPool1d(2)
#         # self.conv3 = nn.Conv1d(2*n_channel, 4 * n_channel, kernel_size=3)
#         # self.bn3 = nn.BatchNorm1d(4 * n_channel)
#         # self.pool3 = nn.MaxPool1d(3)
#         self.lstm = nn.LSTM(2*n_channel, 2*n_channel, num_layers=1, batch_first=True, bidirectional=True)
#         self.dropout = nn.Dropout(0.2)
#         self.fc1 = nn.Linear(4 * n_channel, n_output)

#     def forward(self, x):
#         x = torch.permute(x, [0, 2, 1])
#         x = F.relu(self.bn1(self.cnn1(x)))
#         x = self.pool1(x)
#         x = F.relu(self.bn2(self.cnn2(x)))
#         x = self.pool2(x)
#         # x = F.relu(self.bn3(self.conv3(x)))
#         # x = self.pool3(x)
#         x = self.dropout(x)
#         x = torch.permute(x, [0, 2, 1])
#         x, (h_n, c_n) = self.lstm(x)
#         x = x[:, -1, :]
#         x = x.squeeze(1)
#         x = self.fc1(x)
#         return F.log_softmax(x, dim=1)

# model = CNN(n_input=16, n_output=len(labels))
# model.to(device)
# print(model)

# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# n = count_parameters(model)
# print("Number of parameters: %s" % n)

In [ ]:
# Attach a logger to log model's gradients distribution
# tb_logger.attach(
#     trainer,
#     event_name=Events.EPOCH_COMPLETED,
#     log_handler=GradsHistHandler(model)
# )